In [1]:
# ============================================================
# CELL 0: Imports & Repro
# ============================================================
import os, re, json, math, random
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import (
    accuracy_score, f1_score, cohen_kappa_score, confusion_matrix,
    classification_report, roc_auc_score, average_precision_score
)
from sklearn.preprocessing import label_binarize

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

print("? Imports done.")


? Imports done.


In [2]:
# ============================================================
# CELL 1: Device
# ============================================================
os.environ["CUDA_VISIBLE_DEVICES"] = "2"  # change if needed

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("CUDA:", torch.cuda.is_available(), "| count:", torch.cuda.device_count(), "| device:", device)
if device.type == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))


CUDA: True | count: 1 | device: cuda
GPU: NVIDIA RTX A6000


In [5]:
# ============================================================
# CELL 2: Constants & Paths
# ============================================================
ROOT = Path("/data2/Akbar1/sleep_stages_Dibatic/shhs_sleepstaging_planA/")
MANIFEST_PATH = ROOT / "manifest_sleepstaging_planA.csv"

CKPT_DIR = ROOT / "checkpoints_hier_rope_seq_v5_1"
BEST_CKPT = CKPT_DIR / "BEST_VAL_macroF1.pt"

SUBGROUP_DIR = ROOT / "subgroups_shhs2_server"  # <-- adjust if your folder name differs

LABELS = {0:"W", 1:"N1", 2:"N2", 3:"N3", 4:"REM"}
NUM_CLASSES = 5
FS = 125
T = 3750

assert MANIFEST_PATH.exists(), f"Missing manifest: {MANIFEST_PATH}"
assert CKPT_DIR.exists(), f"Missing ckpt dir: {CKPT_DIR}"
assert BEST_CKPT.exists(), f"Missing best ckpt: {BEST_CKPT}"
assert SUBGROUP_DIR.exists(), f"Missing subgroup dir: {SUBGROUP_DIR}"

print("? Paths OK")
print("Checkpoint:", BEST_CKPT)
print("Subgroup folder:", SUBGROUP_DIR)


? Paths OK
Checkpoint: /data2/Akbar1/sleep_stages_Dibatic/shhs_sleepstaging_planA/checkpoints_hier_rope_seq_v5_1/BEST_VAL_macroF1.pt
Subgroup folder: /data2/Akbar1/sleep_stages_Dibatic/shhs_sleepstaging_planA/subgroups_shhs2_server


In [6]:
# ============================================================
# CELL 3: Load manifest and base SHHS2 external df
# ============================================================
manifest = pd.read_csv(MANIFEST_PATH)

df_ext = manifest[(manifest.cohort=="SHHS2") & (manifest.split=="external_test")].copy()
print("SHHS2 external rows:", len(df_ext))

# sanity
assert "npz_path" in df_ext.columns, "manifest must include npz_path column"


SHHS2 external rows: 2536


In [7]:
# ============================================================
# CELL 4: Normalization + soft-target + duration (same logic)
# ============================================================
def normalize_epochs_zscore(x, eps=1e-6, clip=10.0):
    mu = np.mean(x, axis=1, keepdims=True)
    sd = np.std(x, axis=1, keepdims=True) + eps
    x = (x - mu) / sd
    if clip is not None:
        x = np.clip(x, -clip, clip)
    return x.astype(np.float32)

def _compute_runlength_remaining(y):
    E = len(y)
    rem = np.zeros((E,), dtype=np.int64)
    t = 0
    while t < E:
        j = t
        while j < E and y[j] == y[t]:
            j += 1
        for k in range(t, j):
            rem[k] = (j - k)
        t = j
    return rem

def _bucketize_remaining(rem, edges=(2, 5, 10, 20, 40, 80, 160)):
    edges = np.array(edges, dtype=np.int64)
    b = np.zeros_like(rem, dtype=np.int64)
    for i, r in enumerate(rem):
        b[i] = int(np.searchsorted(edges, r, side="right"))
    return b

def _make_soft_targets_boundary(y, num_classes=5, radius=2, alpha_max=0.35):
    E = len(y)
    soft = np.zeros((E, num_classes), dtype=np.float32)
    soft[np.arange(E), y] = 1.0
    if E < 2 or radius <= 0:
        return soft
    trans = np.where(y[1:] != y[:-1])[0] + 1
    if trans.size == 0:
        return soft
    for t in trans:
        for dt in range(-radius, radius+1):
            i = t + dt
            if i < 0 or i >= E:
                continue
            dist = abs(dt)
            alpha = alpha_max * (1.0 - (dist / max(radius, 1)))
            if alpha <= 0:
                continue
            nb = y[t] if i < t else y[t-1]
            cur = y[i]
            if nb == cur:
                continue
            soft[i, :] = 0.0
            soft[i, cur] = 1.0 - alpha
            soft[i, nb]  = alpha
    soft = soft / (soft.sum(axis=1, keepdims=True) + 1e-8)
    return soft.astype(np.float32)

DUR_EDGES = (2,5,10,20,40,80,160)
DUR_BINS = len(DUR_EDGES) + 1

print("? dataset utilities ready.")


? dataset utilities ready.


In [8]:
# ============================================================
# CELL 5: Dataset + collate_pad (evaluation)
# ============================================================
class SleepSequenceDataset(Dataset):
    def __init__(
        self, df, mode="eval",
        exclude_unknown=True, do_normalize=True,
        soft_radius=2, soft_alpha_max=0.35,
        dur_edges=(2,5,10,20,40,80,160),
        max_hours=None, min_hours=2.0
    ):
        self.paths = df["npz_path"].tolist()
        self.mode = mode
        self.exclude_unknown = exclude_unknown
        self.do_normalize = do_normalize
        self.soft_radius = int(soft_radius)
        self.soft_alpha_max = float(soft_alpha_max)
        self.dur_edges = tuple(dur_edges)
        self.max_hours = max_hours
        self.min_hours = min_hours
        print(f"SleepSequenceDataset[{mode}] files={len(self.paths)} max_hours={self.max_hours} normalize={self.do_normalize}")

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        p = self.paths[idx]
        d = np.load(p, allow_pickle=True)
        x = d["x"].astype(np.float32)  # (E,T)
        y = d["y"].astype(np.int64)    # (E,)
        if self.exclude_unknown:
            keep = (y >= 0)
            x = x[keep]
            y = y[keep]
        if self.do_normalize:
            x = normalize_epochs_zscore(x)

        soft = _make_soft_targets_boundary(y, num_classes=NUM_CLASSES,
                                           radius=self.soft_radius, alpha_max=self.soft_alpha_max)
        rem = _compute_runlength_remaining(y)
        dur_bucket = _bucketize_remaining(rem, edges=self.dur_edges)

        x_t = torch.from_numpy(x).unsqueeze(1)          # (L,1,T)
        y_t = torch.from_numpy(y).long()                # (L,)
        soft_t = torch.from_numpy(soft).float()         # (L,C)
        dur_t  = torch.from_numpy(dur_bucket).long()    # (L,)
        mask = torch.ones((len(y),), dtype=torch.bool)
        return x_t, y_t, mask, soft_t, dur_t

def collate_pad(batch):
    lengths = [b[0].shape[0] for b in batch]
    Lmax = max(lengths)
    xs, ys, ms, ss, ds = [], [], [], [], []
    for x, y, m, s, d in batch:
        L = x.shape[0]
        padL = Lmax - L
        if padL > 0:
            x = torch.cat([x, torch.zeros((padL, 1, T), dtype=x.dtype)], dim=0)
            y = torch.cat([y, torch.zeros((padL,), dtype=y.dtype)], dim=0)
            m = torch.cat([m, torch.zeros((padL,), dtype=torch.bool)], dim=0)
            s = torch.cat([s, torch.zeros((padL, NUM_CLASSES), dtype=s.dtype)], dim=0)
            d = torch.cat([d, torch.zeros((padL,), dtype=d.dtype)], dim=0)
        xs.append(x); ys.append(y); ms.append(m); ss.append(s); ds.append(d)

    x = torch.stack(xs, dim=0)  # (B,L,1,T)
    y = torch.stack(ys, dim=0)  # (B,L)
    m = torch.stack(ms, dim=0)  # (B,L)
    s = torch.stack(ss, dim=0)  # (B,L,C)
    d = torch.stack(ds, dim=0)  # (B,L)
    return x, y, m, s, d

print("? Dataset + collate_pad ready.")


? Dataset + collate_pad ready.


In [9]:
# ============================================================
# CELL 6: Model (HierSleepTransformerV5_1) exact definition
# ============================================================
class DropPath(nn.Module):
    def __init__(self, drop_prob=0.1):
        super().__init__()
        self.drop_prob = float(drop_prob)
    def forward(self, x):
        if (not self.training) or self.drop_prob == 0.0:
            return x
        keep = 1.0 - self.drop_prob
        shape = (x.shape[0],) + (1,) * (x.ndim - 1)
        rand = keep + torch.rand(shape, device=x.device)
        mask = torch.floor(rand)
        return x / keep * mask

class ResConv1D(nn.Module):
    def __init__(self, c_in, c_out, k, s=1):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(c_in, c_out, k, stride=s, padding=k//2),
            nn.BatchNorm1d(c_out),
            nn.GELU(),
            nn.Conv1d(c_out, c_out, k, padding=k//2),
            nn.BatchNorm1d(c_out),
        )
        self.skip = nn.Conv1d(c_in, c_out, 1, stride=s) if (c_in != c_out or s != 1) else nn.Identity()
        self.act = nn.GELU()
    def forward(self, x):
        return self.act(self.conv(x) + self.skip(x))

class EpochEncoder(nn.Module):
    def __init__(self, d_model=384):
        super().__init__()
        self.branch_short = ResConv1D(1, 128, k=7,  s=4)
        self.branch_mid   = ResConv1D(1, 128, k=15, s=4)
        self.branch_long  = ResConv1D(1, 128, k=31, s=4)
        self.freq_proj = nn.Sequential(
            nn.Linear(1876, 256),
            nn.LayerNorm(256),
            nn.GELU(),
        )
        self.fuse = nn.Sequential(
            nn.Linear(128*3 + 256, d_model),
            nn.LayerNorm(d_model),
            nn.GELU(),
        )

    def forward(self, x):
        B, L, _, T_ = x.shape
        x = x.view(B*L, 1, T_)
        zs = self.branch_short(x).mean(-1)
        zm = self.branch_mid(x).mean(-1)
        zl = self.branch_long(x).mean(-1)

        with torch.cuda.amp.autocast(enabled=False):
            xf32 = x.squeeze(1).float()
            Xf = torch.fft.rfft(xf32, dim=-1)
            mag = torch.abs(Xf)
            mag = mag[:, :1876]
            mag = torch.log1p(mag)
            mag = mag / (mag.mean(dim=1, keepdim=True) + 1e-6)
        zf = self.freq_proj(mag)
        z = torch.cat([zs, zm, zl, zf.to(zs.dtype)], dim=-1)
        z = self.fuse(z)
        return z.view(B, L, -1)

def rotate_half(x):
    x1 = x[..., ::2]
    x2 = x[..., 1::2]
    return torch.stack((-x2, x1), dim=-1).flatten(-2)

class RoPE(nn.Module):
    def __init__(self, head_dim, base=10000):
        super().__init__()
        assert head_dim % 2 == 0
        self.head_dim = head_dim
        self.base = base
    def forward(self, x):
        B, L, H, Dh = x.shape
        half = Dh // 2
        freqs = 1.0 / (self.base ** (torch.arange(half, device=x.device) / half))
        t = torch.arange(L, device=x.device)
        angles = torch.einsum("l,d->ld", t, freqs)
        cos = torch.cos(angles)[None, :, None, :]
        sin = torch.sin(angles)[None, :, None, :]
        cos = cos.repeat_interleave(2, dim=-1)
        sin = sin.repeat_interleave(2, dim=-1)
        return (x * cos) + (rotate_half(x) * sin)

def _windows(L, w):
    out, s = [], 0
    while s < L:
        e = min(L, s + w)
        out.append((s, e))
        s = e
    return out

class MultiHeadSelfAttentionRoPE_LocalGlobal(nn.Module):
    def __init__(self, d_model=384, n_heads=8, dropout=0.1, window_size=64):
        super().__init__()
        self.n_heads = n_heads
        self.d_head = d_model // n_heads
        self.window_size = int(window_size)
        self.qkv = nn.Linear(d_model, 3*d_model)
        self.proj = nn.Linear(d_model, d_model)
        self.drop = nn.Dropout(dropout)
        self.rope = RoPE(self.d_head)

    def forward(self, x, key_padding_mask=None, global_attn=False):
        B, L, D = x.shape
        qkv = self.qkv(x)
        q, k, v = qkv.chunk(3, dim=-1)

        q = q.view(B, L, self.n_heads, self.d_head)
        k = k.view(B, L, self.n_heads, self.d_head)
        v = v.view(B, L, self.n_heads, self.d_head)

        q = self.rope(q)
        k = self.rope(k)

        q = q.transpose(1, 2)
        k = k.transpose(1, 2)
        v = v.transpose(1, 2)

        if global_attn or self.window_size >= L:
            scores = (q @ k.transpose(-2, -1)) / math.sqrt(self.d_head)
            scores = scores.float()
            if key_padding_mask is not None:
                scores = scores.masked_fill(~key_padding_mask[:, None, None, :], -1e9)
            attn = torch.softmax(scores, dim=-1)
            attn = self.drop(attn).to(v.dtype)
            out = attn @ v
            out = out.transpose(1, 2).contiguous().view(B, L, D)
            return self.proj(out)

        w = self.window_size
        out = torch.zeros((B, self.n_heads, L, self.d_head), device=x.device, dtype=v.dtype)
        for (s, e) in _windows(L, w):
            qs = q[:, :, s:e, :]
            ks = k[:, :, s:e, :]
            vs = v[:, :, s:e, :]
            scores = (qs @ ks.transpose(-2, -1)) / math.sqrt(self.d_head)
            scores = scores.float()
            if key_padding_mask is not None:
                m = key_padding_mask[:, s:e]
                scores = scores.masked_fill(~m[:, None, None, :], -1e9)
            attn = torch.softmax(scores, dim=-1)
            attn = self.drop(attn).to(vs.dtype)
            out[:, :, s:e, :] = attn @ vs

        out = out.transpose(1, 2).contiguous().view(B, L, D)
        return self.proj(out)

class TransformerBlockLG(nn.Module):
    def __init__(self, d_model=384, n_heads=8, drop=0.1, drop_path=0.1, window_size=64):
        super().__init__()
        self.ln1 = nn.LayerNorm(d_model)
        self.attn = MultiHeadSelfAttentionRoPE_LocalGlobal(d_model, n_heads, drop, window_size=window_size)
        self.ln2 = nn.LayerNorm(d_model)
        self.mlp = nn.Sequential(
            nn.Linear(d_model, 4*d_model),
            nn.GELU(),
            nn.Dropout(drop),
            nn.Linear(4*d_model, d_model),
        )
        self.dp = DropPath(drop_path)

    def forward(self, x, mask, global_attn=False):
        x = x + self.dp(self.attn(self.ln1(x), key_padding_mask=mask, global_attn=global_attn))
        x = x + self.dp(self.mlp(self.ln2(x)))
        return x

class HierSleepTransformerV5_1(nn.Module):
    def __init__(self, num_classes=5, d_model=384, depth=12, n_heads=8, dur_bins=8, window_size=64, global_every=3):
        super().__init__()
        self.num_classes = num_classes
        self.dur_bins = dur_bins
        self.depth = int(depth)
        self.global_every = int(global_every)

        self.encoder = EpochEncoder(d_model)
        self.blocks = nn.ModuleList([
            TransformerBlockLG(
                d_model=d_model, n_heads=n_heads, drop=0.1,
                drop_path=0.1*(i+1)/depth, window_size=window_size
            )
            for i in range(depth)
        ])
        self.head = nn.Linear(d_model, num_classes)
        self.aux_n1 = nn.Linear(d_model, 2)
        self.aux_dur = nn.Linear(d_model, dur_bins)
        self.trans_logits = nn.Parameter(torch.zeros(num_classes, num_classes))

    def forward(self, x, mask):
        z = self.encoder(x)
        for i, blk in enumerate(self.blocks):
            use_global = (self.global_every > 0) and ((i % self.global_every) == 0)
            z = blk(z, mask, global_attn=use_global)
        main_logits = self.head(z)
        aux_logits  = self.aux_n1(z)
        dur_logits  = self.aux_dur(z)
        return main_logits, aux_logits, dur_logits

print("? Model class defined.")


? Model class defined.


In [10]:
# ============================================================
# CELL 7: Load checkpoint + apply EMA weights (exact)
# ============================================================
ckpt = torch.load(BEST_CKPT, map_location="cpu")
print("Checkpoint keys:", list(ckpt.keys())[:20], "...")
print("use_ema:", ckpt.get("use_ema", False), "| has ema_shadow:", isinstance(ckpt.get("ema_shadow", None), dict))

# Build model with same hyperparams as training
model = HierSleepTransformerV5_1(
    num_classes=NUM_CLASSES,
    d_model=384,
    depth=12,
    n_heads=8,
    dur_bins=DUR_BINS,
    window_size=64,
    global_every=3
).to(device)

sd = ckpt["model_state"]
missing, unexpected = model.load_state_dict(sd, strict=False)
print("Loaded weights | missing:", len(missing), "| unexpected:", len(unexpected))

# Apply EMA
use_ema = bool(ckpt.get("use_ema", False))
ema_shadow = ckpt.get("ema_shadow", None)
if use_ema and isinstance(ema_shadow, dict) and len(ema_shadow) > 0:
    curr = model.state_dict()
    applied = 0
    for k, v in ema_shadow.items():
        if k in curr:
            curr[k].copy_(v.to(curr[k].device).to(curr[k].dtype))
            applied += 1
    model.load_state_dict(curr, strict=False)
    print("? Applied EMA tensors:", applied)
else:
    print("?? EMA not applied (missing or disabled).")

model.eval()


Checkpoint keys: ['epoch', 'train_loss', 'model_state', 'optimizer_state', 'val_metrics', 'test_metrics', 'ext_metrics', 'mesa_metrics', 'class_weights', 'use_ema', 'ema_decay', 'ema_shadow', 'use_learned_smoothing', 'use_viterbi', 'Tmat', 'v5_soft_boundary', 'v5_cost_matrix', 'v5_aux_dur', 'best_val_macroF1'] ...
use_ema: True | has ema_shadow: True
Loaded weights | missing: 0 | unexpected: 0
? Applied EMA tensors: 189


HierSleepTransformerV5_1(
  (encoder): EpochEncoder(
    (branch_short): ResConv1D(
      (conv): Sequential(
        (0): Conv1d(1, 128, kernel_size=(7,), stride=(4,), padding=(3,))
        (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): GELU(approximate=none)
        (3): Conv1d(128, 128, kernel_size=(7,), stride=(1,), padding=(3,))
        (4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (skip): Conv1d(1, 128, kernel_size=(1,), stride=(4,))
      (act): GELU(approximate=none)
    )
    (branch_mid): ResConv1D(
      (conv): Sequential(
        (0): Conv1d(1, 128, kernel_size=(15,), stride=(4,), padding=(7,))
        (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): GELU(approximate=none)
        (3): Conv1d(128, 128, kernel_size=(15,), stride=(1,), padding=(7,))
        (4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track

In [11]:
# ============================================================
# CELL 8: Metrics helpers + evaluation that returns EVERYTHING needed
# ============================================================
def _ece_from_probs(y_true, probs, n_bins=15):
    conf = probs.max(axis=1)
    pred = probs.argmax(axis=1)
    acc = (pred == y_true).astype(np.float32)
    bins = np.linspace(0.0, 1.0, n_bins+1)
    ece = 0.0
    for i in range(n_bins):
        lo, hi = bins[i], bins[i+1]
        m = (conf >= lo) & (conf < hi)
        if m.sum() == 0:
            continue
        bin_acc = acc[m].mean()
        bin_conf = conf[m].mean()
        ece += (m.mean()) * abs(bin_acc - bin_conf)
    return float(ece)

def _auroc_auprc_multiclass(y_true, probs, num_classes=5):
    Y = label_binarize(y_true, classes=list(range(num_classes)))
    aurocs, auprcs = [], []
    for c in range(num_classes):
        if Y[:, c].sum() == 0:
            continue
        aurocs.append(roc_auc_score(Y[:, c], probs[:, c]))
        auprcs.append(average_precision_score(Y[:, c], probs[:, c]))
    if len(aurocs) == 0:
        return float("nan"), float("nan")
    return float(np.mean(aurocs)), float(np.mean(auprcs))

# Use learned smoothing if ckpt says so
USE_LEARNED_SMOOTHING = bool(ckpt.get("use_learned_smoothing", True))

def apply_learned_smoothing_probs(probs, model):
    Tm = torch.softmax(model.trans_logits, dim=1)
    return probs @ Tm

@torch.no_grad()
def eval_sequence_full(model, loader, desc="Eval"):
    model.eval()
    y_true_all, y_pred_all, probs_all = [], [], []
    total_n = 0

    for (xb, yb, mb, sb, db) in tqdm(loader, desc=desc, leave=False):
        xb = xb.to(device, non_blocking=True)
        yb = yb.to(device, non_blocking=True)
        mb = mb.to(device, non_blocking=True)

        main_logits, aux_logits, dur_logits = model(xb, mb)
        probs = torch.softmax(main_logits.float(), dim=-1)

        if USE_LEARNED_SMOOTHING:
            probs = apply_learned_smoothing_probs(probs, model)

        pred = torch.argmax(probs, dim=-1)

        yv = yb[mb].detach().cpu().numpy()
        pv = pred[mb].detach().cpu().numpy()
        pr = probs[mb].detach().cpu().numpy()

        if yv.size == 0:
            continue

        y_true_all.append(yv)
        y_pred_all.append(pv)
        probs_all.append(pr)
        total_n += yv.size

    if total_n == 0:
        raise RuntimeError(f"{desc}: collected 0 valid epochs")

    y_true = np.concatenate(y_true_all)
    y_pred = np.concatenate(y_pred_all)
    probs  = np.concatenate(probs_all)

    acc = accuracy_score(y_true, y_pred)
    mf1 = f1_score(y_true, y_pred, average="macro")
    kappa = cohen_kappa_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred, labels=list(range(NUM_CLASSES)))

    auroc, auprc = _auroc_auprc_multiclass(y_true, probs, num_classes=NUM_CLASSES)
    mean_conf = float(probs.max(axis=1).mean())
    ece = _ece_from_probs(y_true, probs, n_bins=15)

    report = classification_report(
        y_true, y_pred,
        labels=list(range(NUM_CLASSES)),
        target_names=[LABELS[i] for i in range(NUM_CLASSES)],
        digits=4,
        output_dict=True,
        zero_division=0
    )

    return dict(
        n_epochs=int(total_n),
        acc=float(acc),
        macro_f1=float(mf1),
        kappa=float(kappa),
        AUROC=float(auroc),
        AUPRC=float(auprc),
        meanConf=float(mean_conf),
        ECE=float(ece),
        cm=cm,
        report=report,
        y_true=y_true,
        y_pred=y_pred,
        probs=probs
    )

print("? eval_sequence_full ready.")


? eval_sequence_full ready.


In [12]:
# ============================================================
# CELL 9: Subgroup runner (no shortcuts, clean)
# ============================================================
subgroup_csvs = sorted(SUBGROUP_DIR.glob("*.csv"))
print("Subgroups found:", len(subgroup_csvs))
assert len(subgroup_csvs) > 0, "No subgroup csv files found."

def parse_axis_group(name):
    base = name.replace(".csv", "")
    base = re.sub(r"^shhs2_", "", base)
    parts = base.split("_", 1)
    axis = parts[0]
    group = parts[1] if len(parts) > 1 else "ALL"
    return axis, group

OUT_DIR = ROOT / "subgroup_eval_outputs"
OUT_DIR.mkdir(exist_ok=True, parents=True)

summary_rows = []

for csv_path in subgroup_csvs:
    axis, group = parse_axis_group(csv_path.name)
    df_sub = pd.read_csv(csv_path)

    # build dataset+loader (batch_size=1 for safety, deterministic)
    ds = SleepSequenceDataset(df_sub, mode="eval", do_normalize=True, soft_radius=2, soft_alpha_max=0.35, dur_edges=DUR_EDGES)
    loader = DataLoader(ds, batch_size=1, shuffle=False, num_workers=1, pin_memory=True, collate_fn=collate_pad)

    res = eval_sequence_full(model, loader, desc=f"SHHS2::{axis}::{group}")

    # save per subgroup json (includes cm + report + metrics)
    save_obj = {
        "axis": axis, "group": group,
        "N_subjects": int(len(df_sub)),
        "n_epochs": int(res["n_epochs"]),
        "metrics": {k: res[k] for k in ["acc","macro_f1","kappa","AUROC","AUPRC","meanConf","ECE"]},
        "confusion_matrix": res["cm"].tolist(),
        "classification_report": res["report"],
    }
    out_json = OUT_DIR / f"SHHS2__{axis}__{group}.json"
    with open(out_json, "w") as f:
        json.dump(save_obj, f, indent=2)

    # add summary
    summary_rows.append({
        "axis": axis, "group": group,
        "N_subjects": int(len(df_sub)),
        "n_epochs": int(res["n_epochs"]),
        "acc": res["acc"],
        "macro_f1": res["macro_f1"],
        "kappa": res["kappa"],
        "AUROC": res["AUROC"],
        "AUPRC": res["AUPRC"],
        "ECE": res["ECE"],
        "meanConf": res["meanConf"],
        "json": out_json.name
    })

    # print block (human readable)
    print("\n" + "="*90)
    print(f"SHHS2 SUBGROUP | axis={axis} | group={group} | subjects={len(df_sub)} | epochs={res['n_epochs']}")
    print(f"acc={res['acc']:.4f}  macroF1={res['macro_f1']:.4f}  kappa={res['kappa']:.4f}  AUROC={res['AUROC']:.4f}  ECE={res['ECE']:.4f}")
    print("Confusion Matrix (rows=true cols=pred):")
    print(res["cm"])
    print("Classification report (macro avg / weighted avg shown):")
    print({k: res["report"][k] for k in ["macro avg","weighted avg"]})
    print("Saved:", out_json)

summary = pd.DataFrame(summary_rows)
summary = summary.sort_values(["axis","group"]).reset_index(drop=True)

out_csv = OUT_DIR / "SHHS2_subgroup_summary.csv"
summary.to_csv(out_csv, index=False)

print("\n? Saved summary:", out_csv)
summary


Subgroups found: 14
SleepSequenceDataset[eval] files=1063 max_hours=None normalize=True



SHHS2 SUBGROUP | axis=age | group=group_adult | subjects=1063 | epochs=1207883
acc=0.8775  macroF1=0.8157  kappa=0.8319  AUROC=0.9751  ECE=0.1741
Confusion Matrix (rows=true cols=pred):
[[327444  15818  16115   4999   6216]
 [  2457  26650   6656    121   4582]
 [  5011  11889 402853  38867  11491]
 [   167     12  11234 130954    170]
 [  1211   4541   6179    190 172056]]
Classification report (macro avg / weighted avg shown):
{'macro avg': {'precision': 0.7935332767849327, 'recall': 0.8504009002278956, 'f1-score': 0.8156576435571538, 'support': 1207883}, 'weighted avg': {'precision': 0.8909097810752001, 'recall': 0.8775328405151823, 'f1-score': 0.8814687426663498, 'support': 1207883}}
Saved: /data2/Akbar1/sleep_stages_Dibatic/shhs_sleepstaging_planA/subgroup_eval_outputs/SHHS2__age__group_adult.json
SleepSequenceDataset[eval] files=1473 max_hours=None normalize=True



SHHS2 SUBGROUP | axis=age | group=group_senior | subjects=1473 | epochs=1651102
acc=0.8636  macroF1=0.7936  kappa=0.8098  AUROC=0.9728  ECE=0.1787
Confusion Matrix (rows=true cols=pred):
[[503394  29595  22204   2145   9087]
 [  4842  39247  16189     32   4842]
 [ 10045  20564 541048  54345  12975]
 [   197     17  16943 145389    148]
 [  2365   6723  11370    545 196851]]
Classification report (macro avg / weighted avg shown):
{'macro avg': {'precision': 0.7724497315306671, 'recall': 0.8270159464453417, 'f1-score': 0.7936218349339035, 'support': 1651102}, 'weighted avg': {'precision': 0.8789619528872834, 'recall': 0.8636225987249728, 'f1-score': 0.8688223213856591, 'support': 1651102}}
Saved: /data2/Akbar1/sleep_stages_Dibatic/shhs_sleepstaging_planA/subgroup_eval_outputs/SHHS2__age__group_senior.json
SleepSequenceDataset[eval] files=922 max_hours=None normalize=True



SHHS2 SUBGROUP | axis=ahi | group=severity_mild | subjects=922 | epochs=1040788
acc=0.8738  macroF1=0.8091  kappa=0.8263  AUROC=0.9749  ECE=0.1765
Confusion Matrix (rows=true cols=pred):
[[299086  15875  11869   2382   5006]
 [  2473  24119   7266     25   3605]
 [  4786  11183 340323  34826   9214]
 [   158     13  10843 105985    133]
 [  1163   4236   6042    216 139961]]
Classification report (macro avg / weighted avg shown):
{'macro avg': {'precision': 0.7873507180101909, 'recall': 0.8432627924358502, 'f1-score': 0.8090501444937337, 'support': 1040788}, 'weighted avg': {'precision': 0.8879362315084747, 'recall': 0.8738321348824161, 'f1-score': 0.8783283289821343, 'support': 1040788}}
Saved: /data2/Akbar1/sleep_stages_Dibatic/shhs_sleepstaging_planA/subgroup_eval_outputs/SHHS2__ahi__severity_mild.json
SleepSequenceDataset[eval] files=614 max_hours=None normalize=True



SHHS2 SUBGROUP | axis=ahi | group=severity_moderate | subjects=614 | epochs=687636
acc=0.8627  macroF1=0.7921  kappa=0.8089  AUROC=0.9726  ECE=0.1743
Confusion Matrix (rows=true cols=pred):
[[201120  11740   9939   1559   4231]
 [  1711  15629   6103     13   2357]
 [  3780   8292 229020  23549   6279]
 [    84      8   6458  60572     86]
 [   878   2887   4390     83  86868]]
Classification report (macro avg / weighted avg shown):
{'macro avg': {'precision': 0.7691194396560425, 'recall': 0.8290573773935701, 'f1-score': 0.7920840617294184, 'support': 687636}, 'weighted avg': {'precision': 0.8792787805264536, 'recall': 0.8626788009935489, 'f1-score': 0.8680141703552913, 'support': 687636}}
Saved: /data2/Akbar1/sleep_stages_Dibatic/shhs_sleepstaging_planA/subgroup_eval_outputs/SHHS2__ahi__severity_moderate.json
SleepSequenceDataset[eval] files=605 max_hours=None normalize=True


...


SHHS2 SUBGROUP | axis=diabetes | group=non-diabetes | subjects=1161 | epochs=1305104
acc=0.8751  macroF1=0.8102  kappa=0.8280  AUROC=0.9740  ECE=0.1773
Confusion Matrix (rows=true cols=pred):
[[370609  19053  16056   3072   6541]
 [  3103  29639  10048    136   4548]
 [  6010  13809 431958  41880  11149]
 [   172     10  12855 134787    148]
 [  1674   5083   7361    235 175168]]
Classification report (macro avg / weighted avg shown):
{'macro avg': {'precision': 0.7895921490742988, 'recall': 0.8414992155505432, 'f1-score': 0.8101635486706555, 'support': 1305104}, 'weighted avg': {'precision': 0.8879884573246865, 'recall': 0.8751494133800831, 'f1-score': 0.8792387457010828, 'support': 1305104}}
Saved: /data2/Akbar1/sleep_stages_Dibatic/shhs_sleepstaging_planA/subgroup_eval_outputs/SHHS2__diabetes__non-diabetes.json
SleepSequenceDataset[eval] files=2536 max_hours=None normalize=True



SHHS2 SUBGROUP | axis=hypertension | group=unknown | subjects=2536 | epochs=2858985
acc=0.8695  macroF1=0.8029  kappa=0.8194  AUROC=0.9739  ECE=0.1767
Confusion Matrix (rows=true cols=pred):
[[830838  45413  38319   7144  15303]
 [  7299  65897  22845    153   9424]
 [ 15056  32453 943901  93212  24466]
 [   364     29  28177 276343    318]
 [  3576  11264  17549    735 368907]]
Classification report (macro avg / weighted avg shown):
{'macro avg': {'precision': 0.7812247798803751, 'recall': 0.8369256144241373, 'f1-score': 0.8028968102273547, 'support': 2858985}, 'weighted avg': {'precision': 0.8839755247127055, 'recall': 0.8694994902036912, 'f1-score': 0.8741586375566678, 'support': 2858985}}
Saved: /data2/Akbar1/sleep_stages_Dibatic/shhs_sleepstaging_planA/subgroup_eval_outputs/SHHS2__hypertension__unknown.json
SleepSequenceDataset[eval] files=1375 max_hours=None normalize=True



SHHS2 SUBGROUP | axis=sex | group=female | subjects=1375 | epochs=1561632
acc=0.8784  macroF1=0.8148  kappa=0.8340  AUROC=0.9774  ECE=0.1790
Confusion Matrix (rows=true cols=pred):
[[445824  20435  16638   3244   7319]
 [  3680  31739   9617     16   4777]
 [  7598  15077 490723  54357  13317]
 [   254     18  16976 193047    242]
 [  1441   5055   9251    607 210380]]
Classification report (macro avg / weighted avg shown):
{'macro avg': {'precision': 0.7947087219197153, 'recall': 0.8459470222149719, 'f1-score': 0.8148481555401712, 'support': 1561632}, 'weighted avg': {'precision': 0.8901887611728536, 'recall': 0.8783842800352452, 'f1-score': 0.88200581954493, 'support': 1561632}}
Saved: /data2/Akbar1/sleep_stages_Dibatic/shhs_sleepstaging_planA/subgroup_eval_outputs/SHHS2__sex__female.json
SleepSequenceDataset[eval] files=1161 max_hours=None normalize=True



SHHS2 SUBGROUP | axis=sex | group=male | subjects=1161 | epochs=1297353
acc=0.8588  macroF1=0.7846  kappa=0.8003  AUROC=0.9695  ECE=0.1739
Confusion Matrix (rows=true cols=pred):
[[385014  24978  21681   3900   7984]
 [  3619  34158  13228    137   4647]
 [  7458  17376 453178  38855  11149]
 [   110     11  11201  83296     76]
 [  2135   6209   8298    128 158527]]
Classification report (macro avg / weighted avg shown):
{'macro avg': {'precision': 0.7601730732088341, 'recall': 0.8245037472704795, 'f1-score': 0.7846422876430512, 'support': 1297353}, 'weighted avg': {'precision': 0.8771599888720607, 'recall': 0.8588048125683604, 'f1-score': 0.8649081291594299, 'support': 1297353}}
Saved: /data2/Akbar1/sleep_stages_Dibatic/shhs_sleepstaging_planA/subgroup_eval_outputs/SHHS2__sex__male.json

? Saved summary: /data2/Akbar1/sleep_stages_Dibatic/shhs_sleepstaging_planA/subgroup_eval_outputs/SHHS2_subgroup_summary.csv


,axis,group,N_subjects,n_epochs,acc,macro_f1,kappa,AUROC,AUPRC,ECE,meanConf,json
0,age,group_adult,1063,1207883,0.877533,0.815658,0.831897,0.975139,0.873464,0.174094,0.703443,SHHS2__age__group_adult.json
1,age,group_senior,1473,1651102,0.863623,0.793622,0.809810,0.972795,0.852270,0.178655,0.684968,SHHS2__age__group_senior.json
2,ahi,severity_mild,922,1040788,0.873832,0.809050,0.826325,0.974877,0.865266,0.176518,0.697321,SHHS2__ahi__severity_mild.json
3,ahi,severity_moderate,614,687636,0.862679,0.792084,0.808918,0.972587,0.851966,0.174251,0.688428,SHHS2__ahi__severity_moderate.json
4,ahi,severity_no_osa,605,684726,0.885007,0.821613,0.842588,0.979888,0.879332,0.178908,0.706099,SHHS2__ahi__severity_no_osa.json
5,ahi,severity_severe,395,445835,0.846089,0.774545,0.780990,0.964017,0.838028,0.177701,0.668395,SHHS2__ahi__severity_severe.json
6,bmi,group_normal,649,730333,0.874067,0.808613,0.826620,0.976337,0.866730,0.178693,0.695385,SHHS2__bmi__group_normal.json
7,bmi,group_obese,807,913767,0.866841,0.799424,0.814909,0.972834,0.858455,0.178007,0.688842,SHHS2__bmi__group_obese.json
8,bmi,group_overweight,1059,1191533,0.868955,0.802246,0.818597,0.973398,0.861031,0.174603,0.694352,SHHS2__bmi__group_overweight.json
9,diabetes,diabetes,1375,1553881,0.864754,0.796655,0.812044,0.973856,0.857592,0.176214,0.688545,SHHS2__diabetes__diabetes.json


In [13]:
# ============================================================
# CELL 10: Pretty table (paper-ready)
# ============================================================
def pretty(df):
    return (df.style
            .format({
                "acc":"{:.4f}", "macro_f1":"{:.4f}", "kappa":"{:.4f}",
                "AUROC":"{:.4f}", "AUPRC":"{:.4f}", "ECE":"{:.4f}", "meanConf":"{:.4f}"
            })
            .set_properties(**{"text-align": "center"})
           )

pretty(summary)


,axis,group,N_subjects,n_epochs,acc,macro_f1,kappa,AUROC,AUPRC,ECE,meanConf,json
0,age,group_adult,1063,1207883,0.8775,0.8157,0.8319,0.9751,0.8735,0.1741,0.7034,SHHS2__age__group_adult.json
1,age,group_senior,1473,1651102,0.8636,0.7936,0.8098,0.9728,0.8523,0.1787,0.6850,SHHS2__age__group_senior.json
2,ahi,severity_mild,922,1040788,0.8738,0.8091,0.8263,0.9749,0.8653,0.1765,0.6973,SHHS2__ahi__severity_mild.json
3,ahi,severity_moderate,614,687636,0.8627,0.7921,0.8089,0.9726,0.8520,0.1743,0.6884,SHHS2__ahi__severity_moderate.json
4,ahi,severity_no_osa,605,684726,0.8850,0.8216,0.8426,0.9799,0.8793,0.1789,0.7061,SHHS2__ahi__severity_no_osa.json
5,ahi,severity_severe,395,445835,0.8461,0.7745,0.7810,0.9640,0.8380,0.1777,0.6684,SHHS2__ahi__severity_severe.json
6,bmi,group_normal,649,730333,0.8741,0.8086,0.8266,0.9763,0.8667,0.1787,0.6954,SHHS2__bmi__group_normal.json
7,bmi,group_obese,807,913767,0.8668,0.7994,0.8149,0.9728,0.8585,0.1780,0.6888,SHHS2__bmi__group_obese.json
8,bmi,group_overweight,1059,1191533,0.8690,0.8022,0.8186,0.9734,0.8610,0.1746,0.6944,SHHS2__bmi__group_overweight.json
9,diabetes,diabetes,1375,1553881,0.8648,0.7967,0.8120,0.9739,0.8576,0.1762,0.6885,SHHS2__diabetes__diabetes.json


In [14]:
# ============================================================
# CELL 11: ROC + Reliability plots per subgroup (saved as PNGs)
# ============================================================
import matplotlib.pyplot as plt

PLOT_DIR = OUT_DIR / "plots"
PLOT_DIR.mkdir(exist_ok=True, parents=True)

def _safe_name(s):
    s = str(s)
    s = re.sub(r"[^\w\-_\.]+", "_", s)
    return s[:180]

def plot_ovr_roc(y_true, probs, title, save_path):
    """
    One-vs-rest ROC for multiclass. Plots only classes present in y_true.
    """
    classes = list(range(NUM_CLASSES))
    Y = label_binarize(y_true, classes=classes)  # (N,C)

    plt.figure(figsize=(7.5, 6))
    any_curve = False
    for c in classes:
        if Y[:, c].sum() == 0:
            continue
        # ROC curve points manually (no sklearn roc_curve to keep dependencies minimal)
        # We'll use threshold sweep via sorting
        scores = probs[:, c]
        yb = Y[:, c].astype(np.int32)

        order = np.argsort(-scores)
        yb = yb[order]
        scores_sorted = scores[order]

        P = yb.sum()
        N = len(yb) - P
        if P == 0 or N == 0:
            continue

        tps = np.cumsum(yb)
        fps = np.cumsum(1 - yb)
        tpr = tps / max(P, 1)
        fpr = fps / max(N, 1)

        # add (0,0) at start
        fpr = np.concatenate([[0.0], fpr])
        tpr = np.concatenate([[0.0], tpr])

        auc_c = roc_auc_score(Y[:, c], probs[:, c])
        plt.plot(fpr, tpr, label=f"{LABELS[c]} (AUC={auc_c:.3f})")
        any_curve = True

    plt.plot([0, 1], [0, 1], linestyle="--")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(title)
    plt.legend(loc="lower right", fontsize=9)
    plt.tight_layout()

    if any_curve:
        plt.savefig(save_path, dpi=200)
    plt.close()

def plot_reliability(y_true, probs, title, save_path, n_bins=15):
    """
    Reliability diagram (top-1 confidence vs accuracy).
    """
    conf = probs.max(axis=1)
    pred = probs.argmax(axis=1)
    acc = (pred == y_true).astype(np.float32)

    bins = np.linspace(0.0, 1.0, n_bins + 1)
    bin_centers = (bins[:-1] + bins[1:]) / 2.0

    bin_accs = np.zeros(n_bins, dtype=np.float32)
    bin_confs = np.zeros(n_bins, dtype=np.float32)
    bin_counts = np.zeros(n_bins, dtype=np.int32)

    for i in range(n_bins):
        lo, hi = bins[i], bins[i+1]
        m = (conf >= lo) & (conf < hi)
        bin_counts[i] = int(m.sum())
        if bin_counts[i] > 0:
            bin_accs[i] = acc[m].mean()
            bin_confs[i] = conf[m].mean()
        else:
            bin_accs[i] = np.nan
            bin_confs[i] = np.nan

    plt.figure(figsize=(7.5, 6))
    # plot only non-empty bins
    m = ~np.isnan(bin_accs)
    plt.plot([0, 1], [0, 1], linestyle="--", label="Perfect calibration")
    plt.plot(bin_confs[m], bin_accs[m], marker="o", label="Model")

    # add a small histogram panel via scaled bars (no subplots rule isn't relevant here since single axes)
    # We'll show counts as faint bars on same axis using secondary scale idea (simple)
    maxc = max(bin_counts.max(), 1)
    heights = (bin_counts / maxc) * 0.2  # scaled to 0..0.2 range
    plt.bar(bin_centers, heights, width=(1.0/n_bins)*0.9, alpha=0.25, align="center",
            label="Bin counts (scaled)")

    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel("Confidence (top-1)")
    plt.ylabel("Accuracy")
    plt.title(title)
    plt.legend(loc="lower right", fontsize=9)
    plt.tight_layout()
    plt.savefig(save_path, dpi=200)
    plt.close()

# If you want plots from the in-memory `summary` table, iterate through those jsons:
json_paths = sorted(OUT_DIR.glob("SHHS2__*.json"))
print("Found subgroup JSONs:", len(json_paths))

for jp in tqdm(json_paths, desc="Plotting subgroups"):
    with open(jp, "r") as f:
        obj = json.load(f)
    axis = obj["axis"]
    group = obj["group"]

    # To plot, we need y_true + probs.
    # If you also want to store y_true/probs, easiest is to re-run inference here.
    # We'll re-run inference because JSON stores only metrics + CM + report.
    csv_guess = SUBGROUP_DIR / f"shhs2_{axis}_{group}.csv"
    if not csv_guess.exists():
        # fallback: find any csv that matches axis/group
        hits = [p for p in subgroup_csvs if (parse_axis_group(p.name) == (axis, group))]
        if len(hits) == 0:
            print("?? Could not locate CSV for:", axis, group, "| skipping plots")
            continue
        csv_guess = hits[0]

    df_sub = pd.read_csv(csv_guess)
    ds = SleepSequenceDataset(df_sub, mode="eval", do_normalize=True, soft_radius=2, soft_alpha_max=0.35, dur_edges=DUR_EDGES)
    loader = DataLoader(ds, batch_size=1, shuffle=False, num_workers=1, pin_memory=True, collate_fn=collate_pad)

    # run inference (fast, eval only)
    res = eval_sequence_full(model, loader, desc=f"PLOT::{axis}::{group}")
    y_true = res["y_true"]
    probs = res["probs"]

    roc_path = PLOT_DIR / f"roc__SHHS2__{_safe_name(axis)}__{_safe_name(group)}.png"
    rel_path = PLOT_DIR / f"reliability__SHHS2__{_safe_name(axis)}__{_safe_name(group)}.png"

    plot_ovr_roc(
        y_true, probs,
        title=f"SHHS2 ROC (OvR) | {axis}={group} | N={len(y_true)}",
        save_path=roc_path
    )
    plot_reliability(
        y_true, probs,
        title=f"SHHS2 Reliability | {axis}={group} | N={len(y_true)}",
        save_path=rel_path,
        n_bins=15
    )

print("\n? Saved plots to:", PLOT_DIR)
sorted(PLOT_DIR.glob("*.png"))[:10]


Found subgroup JSONs: 14


Plotting subgroups:   0%|                                | 0/14 [00:00<?, ?it/s]

SleepSequenceDataset[eval] files=1063 max_hours=None normalize=True



PLOT::age::group_adult:  18%|██▉             | 193/1063 [00:43<03:41,  3.93it/s]


PLOT::age::group_adult:  36%|█████▊          | 387/1063 [01:27<02:17,  4.91it/s]


...

PLOT::sex::male:  67%|███████████████▎       | 775/1161 [02:50<01:24,  4.55it/s]


PLOT::sex::male:  83%|███████████████████▏   | 969/1161 [03:31<00:39,  4.84it/s]


Plotting subgroups: 100%|██████████████████████| 14/14 [57:10<00:00, 245.03s/it]


? Saved plots to: /data2/Akbar1/sleep_stages_Dibatic/shhs_sleepstaging_planA/subgroup_eval_outputs/plots


[PosixPath('/data2/Akbar1/sleep_stages_Dibatic/shhs_sleepstaging_planA/subgroup_eval_outputs/plots/reliability__SHHS2__age__group_adult.png'),
 PosixPath('/data2/Akbar1/sleep_stages_Dibatic/shhs_sleepstaging_planA/subgroup_eval_outputs/plots/reliability__SHHS2__age__group_senior.png'),
 PosixPath('/data2/Akbar1/sleep_stages_Dibatic/shhs_sleepstaging_planA/subgroup_eval_outputs/plots/reliability__SHHS2__ahi__severity_mild.png'),
 PosixPath('/data2/Akbar1/sleep_stages_Dibatic/shhs_sleepstaging_planA/subgroup_eval_outputs/plots/reliability__SHHS2__ahi__severity_moderate.png'),
 PosixPath('/data2/Akbar1/sleep_stages_Dibatic/shhs_sleepstaging_planA/subgroup_eval_outputs/plots/reliability__SHHS2__ahi__severity_no_osa.png'),
 PosixPath('/data2/Akbar1/sleep_stages_Dibatic/shhs_sleepstaging_planA/subgroup_eval_outputs/plots/reliability__SHHS2__ahi__severity_severe.png'),
 PosixPath('/data2/Akbar1/sleep_stages_Dibatic/shhs_sleepstaging_planA/subgroup_eval_outputs/plots/reliability__SHHS2__bmi__g

In [21]:
# ============================================================
# CELL M0: Use existing MESA subgroup CSVs (no regeneration)
# ============================================================
from pathlib import Path
import pandas as pd
import numpy as np
import re, json
from tqdm import tqdm

SUBGROUP_DIR = Path("/data2/Akbar1/sleep_stages_Dibatic/shhs_sleepstaging_planA/subgroups_mesa_server")

assert SUBGROUP_DIR.exists(), f"Folder not found: {SUBGROUP_DIR}"

mesa_subgroup_csvs = sorted(SUBGROUP_DIR.glob("mesa_*.csv"))
print("Found subgroup CSVs:", len(mesa_subgroup_csvs))
for p in mesa_subgroup_csvs:
    print(" -", p.name)

# quick peek
pd.read_csv(mesa_subgroup_csvs[0]).head()


Found subgroup CSVs: 13
 - mesa_age_group_adult.csv
 - mesa_age_group_senior.csv
 - mesa_ahi_severity_mild.csv
 - mesa_ahi_severity_moderate.csv
 - mesa_ahi_severity_no_osa.csv
 - mesa_ahi_severity_severe.csv
 - mesa_bmi_group_normal.csv
 - mesa_bmi_group_obese.csv
 - mesa_bmi_group_overweight.csv
 - mesa_hypertension_normotensive.csv
 - mesa_hypertension_unknown.csv
 - mesa_sex_female.csv
 - mesa_sex_male.csv


,nsrrid,mesaid,sex,age_group,bmi_group,ahi_severity,hypertension,ahi,avdnbp5,avdrbp5,dataset,npz_path
0,6,6,Female,Adult,Obese,Severe,Normotensive,62.51,4.0,11.0,MESA,/data2/Akbar1/sleep_stages_Dibatic/shhs_sleeps...
1,14,14,Female,Adult,Obese,Mild,Normotensive,11.37,3.0,5.0,MESA,/data2/Akbar1/sleep_stages_Dibatic/shhs_sleeps...
2,16,16,Female,Adult,Normal,No OSA,NaN,2.70,2.0,NaN,MESA,/data2/Akbar1/sleep_stages_Dibatic/shhs_sleeps...
3,28,28,Female,Adult,Obese,Moderate,Normotensive,21.87,2.0,7.0,MESA,/data2/Akbar1/sleep_stages_Dibatic/shhs_sleeps...
4,38,38,Male,Adult,Obese,Mild,NaN,7.33,NaN,NaN,MESA,/data2/Akbar1/sleep_stages_Dibatic/shhs_sleeps...


In [22]:
# ============================================================
# CELL M1: Signature-safe dataset/loader builder (robust)
# ============================================================
import inspect
import torch
from torch.utils.data import DataLoader

def _filter_kwargs_for_callable(fn, kwargs: dict):
    sig = inspect.signature(fn)
    valid = {}
    for k, v in kwargs.items():
        if k in sig.parameters:
            valid[k] = v
    return valid

def make_loader_for_subgroup_csv(csv_path, batch_size=1):
    df_sub = pd.read_csv(csv_path)

    # ---- build kwargs that match YOUR dataset implementation ----
    ds_kwargs = dict(
        df=df_sub,
        mode="eval",
        max_hours=None,
        min_hours=0.0,
        augmentor=None,
        exclude_unknown=True,
        do_normalize=True,
        soft_radius=2,
        soft_alpha_max=0.35,
        dur_edges=DUR_EDGES if "DUR_EDGES" in globals() else (2,5,10,20,40,80,160),
        boundary_oversample_p=0.0,
        boundary_radius=2,
    )

    # Your class signature may differ depending on which cell you executed.
    # So we filter kwargs safely:
    ds_kwargs = _filter_kwargs_for_callable(SleepSequenceDataset.__init__, ds_kwargs)

    ds = SleepSequenceDataset(**ds_kwargs)

    loader = DataLoader(
        ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=0,          # important for stability
        pin_memory=False,       # safest on servers
        collate_fn=collate_pad
    )
    return loader


In [23]:
# ============================================================
# CELL M2: eval_sequence_full (metrics + cm + classification report + probs)
# ============================================================
from sklearn.metrics import (
    accuracy_score, f1_score, cohen_kappa_score,
    confusion_matrix, classification_report,
    roc_auc_score, average_precision_score
)
from sklearn.preprocessing import label_binarize

import numpy as np
import torch

def _ece_from_probs_np(y_true, probs, n_bins=15):
    conf = probs.max(axis=1)
    pred = probs.argmax(axis=1)
    acc  = (pred == y_true).astype(np.float32)

    bins = np.linspace(0.0, 1.0, n_bins+1)
    ece = 0.0
    for i in range(n_bins):
        lo, hi = bins[i], bins[i+1]
        m = (conf >= lo) & (conf < hi)
        if m.sum() == 0:
            continue
        ece += (m.mean()) * abs(acc[m].mean() - conf[m].mean())
    return float(ece)

def _auroc_auprc_multiclass_np(y_true, probs, num_classes):
    Y = label_binarize(y_true, classes=list(range(num_classes)))
    aurocs, auprcs = [], []
    for c in range(num_classes):
        if Y[:, c].sum() == 0:
            continue
        try:
            aurocs.append(roc_auc_score(Y[:, c], probs[:, c]))
            auprcs.append(average_precision_score(Y[:, c], probs[:, c]))
        except Exception:
            pass
    if len(aurocs) == 0:
        return float("nan"), float("nan")
    return float(np.mean(aurocs)), float(np.mean(auprcs))

@torch.no_grad()
def eval_sequence_full(model, loader, desc="Eval"):
    model.eval()

    all_true, all_pred, all_probs = [], [], []
    total_loss = 0.0
    total_n = 0
    bad_batches = 0

    for (xb, yb, mb, sb, db) in tqdm(loader, desc=desc, leave=False):
        xb = xb.to(device, non_blocking=True)
        yb = yb.to(device, non_blocking=True)
        mb = mb.to(device, non_blocking=True)
        sb = sb.to(device, non_blocking=True) if isinstance(sb, torch.Tensor) else sb
        db = db.to(device, non_blocking=True) if isinstance(db, torch.Tensor) else db

        # forward
        with torch.cuda.amp.autocast(enabled=(device.type=="cuda")):
            main_logits, aux_logits, dur_logits = model(xb, mb)

            # use your loss if it exists; otherwise skip loss
            if "masked_loss_v5" in globals():
                loss = masked_loss_v5(model, main_logits, aux_logits, dur_logits, yb, mb, sb, db)
            else:
                loss = torch.zeros((), device=device)

        if (not torch.isfinite(main_logits).all()) or (not torch.isfinite(loss)):
            bad_batches += 1
            continue

        probs = torch.softmax(main_logits.float(), dim=-1)

        # optional learned smoothing if enabled in your notebook
        if "USE_LEARNED_SMOOTHING" in globals() and USE_LEARNED_SMOOTHING and ("apply_learned_smoothing_probs" in globals()):
            probs = apply_learned_smoothing_probs(probs, model)

        pred = torch.argmax(probs, dim=-1)

        yv = yb[mb].detach().cpu().numpy()
        pv = pred[mb].detach().cpu().numpy()
        pr = probs[mb].detach().cpu().numpy()

        if yv.size == 0:
            continue

        all_true.append(yv)
        all_pred.append(pv)
        all_probs.append(pr)

        n = int(mb.sum().item())
        total_loss += float(loss.item()) * n
        total_n += n

    if len(all_true) == 0:
        return dict(
            loss=float("nan"), acc=float("nan"), macro_f1=float("nan"), kappa=float("nan"),
            AUROC=float("nan"), AUPRC=float("nan"), ECE=float("nan"), meanConf=float("nan"),
            cm=np.zeros((NUM_CLASSES, NUM_CLASSES), dtype=np.int64),
            report={},
            f1_per_class={LABELS[i]: float("nan") for i in range(NUM_CLASSES)},
            y_true=np.array([], dtype=np.int64),
            y_pred=np.array([], dtype=np.int64),
            probs=np.zeros((0, NUM_CLASSES), dtype=np.float32),
            bad_batches=int(bad_batches),
        )

    y_true = np.concatenate(all_true)
    y_pred = np.concatenate(all_pred)
    probs  = np.concatenate(all_probs)

    acc   = float(accuracy_score(y_true, y_pred))
    mf1   = float(f1_score(y_true, y_pred, average="macro"))
    kappa = float(cohen_kappa_score(y_true, y_pred))
    cm    = confusion_matrix(y_true, y_pred, labels=list(range(NUM_CLASSES)))

    auroc, auprc = _auroc_auprc_multiclass_np(y_true, probs, NUM_CLASSES)
    ece = _ece_from_probs_np(y_true, probs, n_bins=15)
    mean_conf = float(probs.max(axis=1).mean())

    f1_per = {LABELS[i]: float(f1_score((y_true==i).astype(int), (y_pred==i).astype(int))) for i in range(NUM_CLASSES)}
    rep = classification_report(
        y_true, y_pred,
        labels=list(range(NUM_CLASSES)),
        target_names=[LABELS[i] for i in range(NUM_CLASSES)],
        output_dict=True,
        zero_division=0
    )

    return dict(
        loss=(total_loss / max(total_n,1)),
        acc=acc, macro_f1=mf1, kappa=kappa,
        AUROC=auroc, AUPRC=auprc,
        ECE=ece, meanConf=mean_conf,
        cm=cm, report=rep, f1_per_class=f1_per,
        y_true=y_true, y_pred=y_pred, probs=probs,
        bad_batches=int(bad_batches)
    )


In [24]:
# ============================================================
# CELL M3: Evaluate all existing MESA subgroup CSVs + save outputs
# ============================================================
import matplotlib.pyplot as plt

OUT_DIR = Path("/data2/Akbar1/sleep_stages_Dibatic/shhs_sleepstaging_planA/MESA_subgroup_analysis_outputs")
OUT_DIR.mkdir(parents=True, exist_ok=True)

MESA_RESULTS_DIR = OUT_DIR / "mesa_subgroup_json"
MESA_RESULTS_DIR.mkdir(exist_ok=True, parents=True)

MESA_PLOT_DIR = OUT_DIR / "mesa_subgroup_plots"
MESA_PLOT_DIR.mkdir(exist_ok=True, parents=True)

def _safe_name(s):
    s = str(s)
    s = re.sub(r"[^\w\-_\.]+", "_", s)
    return s[:180]

def parse_axis_group_from_filename(name):
    # expects: mesa_<axis>_<group>.csv  (your current filenames)
    base = name.replace(".csv", "")
    # remove leading mesa_
    base = base[5:] if base.startswith("mesa_") else base
    parts = base.split("_")
    axis = parts[0]
    group = "_".join(parts[1:]) if len(parts) > 1 else "unknown"
    return axis, group

def plot_ovr_roc(y_true, probs, title, save_path):
    classes = list(range(NUM_CLASSES))
    Y = label_binarize(y_true, classes=classes)

    plt.figure(figsize=(7.5, 6))
    any_curve = False

    for c in classes:
        if Y[:, c].sum() == 0:
            continue
        try:
            auc_c = roc_auc_score(Y[:, c], probs[:, c])
        except Exception:
            continue

        scores = probs[:, c]
        yb = Y[:, c].astype(np.int32)
        order = np.argsort(-scores)
        yb = yb[order]

        P = yb.sum()
        N = len(yb) - P
        if P == 0 or N == 0:
            continue

        tps = np.cumsum(yb)
        fps = np.cumsum(1 - yb)
        tpr = tps / max(P, 1)
        fpr = fps / max(N, 1)

        fpr = np.concatenate([[0.0], fpr])
        tpr = np.concatenate([[0.0], tpr])

        plt.plot(fpr, tpr, label=f"{LABELS[c]} (AUC={auc_c:.3f})")
        any_curve = True

    plt.plot([0, 1], [0, 1], linestyle="--")
    plt.xlim([0, 1]); plt.ylim([0, 1])
    plt.xlabel("False Positive Rate"); plt.ylabel("True Positive Rate")
    plt.title(title)
    plt.legend(loc="lower right", fontsize=9)
    plt.tight_layout()
    if any_curve:
        plt.savefig(save_path, dpi=200)
    plt.close()

def plot_reliability(y_true, probs, title, save_path, n_bins=15):
    conf = probs.max(axis=1)
    pred = probs.argmax(axis=1)
    acc = (pred == y_true).astype(np.float32)

    bins = np.linspace(0.0, 1.0, n_bins + 1)
    centers = (bins[:-1] + bins[1:]) / 2.0

    bin_accs = np.full(n_bins, np.nan, dtype=np.float32)
    bin_confs = np.full(n_bins, np.nan, dtype=np.float32)
    bin_counts = np.zeros(n_bins, dtype=np.int32)

    for i in range(n_bins):
        lo, hi = bins[i], bins[i+1]
        m = (conf >= lo) & (conf < hi)
        bin_counts[i] = int(m.sum())
        if bin_counts[i] > 0:
            bin_accs[i] = acc[m].mean()
            bin_confs[i] = conf[m].mean()

    plt.figure(figsize=(7.5, 6))
    plt.plot([0, 1], [0, 1], linestyle="--", label="Perfect calibration")
    m = ~np.isnan(bin_accs)
    plt.plot(bin_confs[m], bin_accs[m], marker="o", label="Model")

    maxc = max(bin_counts.max(), 1)
    heights = (bin_counts / maxc) * 0.2
    plt.bar(centers, heights, width=(1.0/n_bins)*0.9, alpha=0.25, align="center",
            label="Bin counts (scaled)")

    plt.xlim([0, 1]); plt.ylim([0, 1])
    plt.xlabel("Confidence (top-1)")
    plt.ylabel("Accuracy")
    plt.title(title)
    plt.legend(loc="lower right", fontsize=9)
    plt.tight_layout()
    plt.savefig(save_path, dpi=200)
    plt.close()

def plot_cm(cm, title, save_path):
    plt.figure(figsize=(6.5, 5.5))
    plt.imshow(cm, interpolation="nearest")
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(NUM_CLASSES)
    plt.xticks(tick_marks, [LABELS[i] for i in range(NUM_CLASSES)], rotation=45)
    plt.yticks(tick_marks, [LABELS[i] for i in range(NUM_CLASSES)])
    plt.ylabel("True")
    plt.xlabel("Predicted")
    plt.tight_layout()
    plt.savefig(save_path, dpi=200)
    plt.close()

rows = []

print("Evaluating subgroup CSVs:", len(mesa_subgroup_csvs))

for csv_path in tqdm(mesa_subgroup_csvs, desc="MESA subgroup eval"):
    axis, group = parse_axis_group_from_filename(csv_path.name)

    loader = make_loader_for_subgroup_csv(csv_path, batch_size=1)
    res = eval_sequence_full(model, loader, desc=f"MESA::{axis}::{group}")

    # Save JSON
    out_json = MESA_RESULTS_DIR / f"MESA__{_safe_name(axis)}__{_safe_name(group)}.json"
    payload = dict(
        dataset="MESA",
        axis=axis,
        group=group,
        N_epochs=int(res["y_true"].shape[0]),
        loss=float(res["loss"]) if res["loss"]==res["loss"] else None,
        acc=float(res["acc"]) if res["acc"]==res["acc"] else None,
        macro_f1=float(res["macro_f1"]) if res["macro_f1"]==res["macro_f1"] else None,
        kappa=float(res["kappa"]) if res["kappa"]==res["kappa"] else None,
        AUROC=float(res["AUROC"]) if res["AUROC"]==res["AUROC"] else None,
        AUPRC=float(res["AUPRC"]) if res["AUPRC"]==res["AUPRC"] else None,
        ECE=float(res["ECE"]) if res["ECE"]==res["ECE"] else None,
        meanConf=float(res["meanConf"]) if res["meanConf"]==res["meanConf"] else None,
        f1_per_class=res["f1_per_class"],
        cm=res["cm"].tolist(),
        classification_report=res["report"],
        bad_batches=int(res["bad_batches"]),
        source_csv=str(csv_path),
    )
    with open(out_json, "w") as f:
        json.dump(payload, f, indent=2)

    # Plots
    if res["y_true"].size > 0:
        roc_path = MESA_PLOT_DIR / f"roc__{_safe_name(axis)}__{_safe_name(group)}.png"
        rel_path = MESA_PLOT_DIR / f"reliability__{_safe_name(axis)}__{_safe_name(group)}.png"
        cm_path  = MESA_PLOT_DIR / f"cm__{_safe_name(axis)}__{_safe_name(group)}.png"

        plot_ovr_roc(res["y_true"], res["probs"],
                     title=f"MESA ROC (OvR) | {axis}={group} | N={len(res['y_true'])}",
                     save_path=roc_path)

        plot_reliability(res["y_true"], res["probs"],
                         title=f"MESA Reliability | {axis}={group} | N={len(res['y_true'])}",
                         save_path=rel_path,
                         n_bins=15)

        plot_cm(res["cm"],
                title=f"MESA Confusion Matrix | {axis}={group}",
                save_path=cm_path)

    # Summary row
    rows.append(dict(
        dataset="MESA",
        axis=axis,
        group=group,
        N_epochs=int(res["y_true"].shape[0]),
        acc=res["acc"],
        macro_f1=res["macro_f1"],
        kappa=res["kappa"],
        AUROC=res["AUROC"],
        AUPRC=res["AUPRC"],
        ECE=res["ECE"],
        meanConf=res["meanConf"],
        bad_batches=res["bad_batches"],
        csv=str(csv_path),
        json=str(out_json),
    ))

mesa_df = pd.DataFrame(rows).sort_values(["axis","group"]).reset_index(drop=True)
mesa_csv = OUT_DIR / "MESA_subgroup_summary.csv"
mesa_df.to_csv(mesa_csv, index=False)

print("\n? Saved:")
print(" - Summary:", mesa_csv)
print(" - JSON dir:", MESA_RESULTS_DIR)
print(" - Plot dir:", MESA_PLOT_DIR)

mesa_df


Evaluating subgroup CSVs: 13


MESA subgroup eval:   0%|                                | 0/13 [00:00<?, ?it/s]

SleepSequenceDataset[eval] files=700 max_hours=None normalize=True



MESA::age::group_adult:  28%|████▋            | 193/700 [00:45<01:58,  4.27it/s]


MESA::age::group_adult:  55%|█████████▍       | 387/700 [01:31<01:14,  4.22it/s]


...

MESA::sex::male:  70%|████████████████▊       | 581/831 [02:15<00:54,  4.56it/s]


MESA::sex::male:  93%|██████████████████████▍ | 775/831 [03:01<00:13,  4.26it/s]


MESA subgroup eval: 100%|██████████████████████| 13/13 [37:59<00:00, 175.31s/it]


? Saved:
 - Summary: /data2/Akbar1/sleep_stages_Dibatic/shhs_sleepstaging_planA/MESA_subgroup_analysis_outputs/MESA_subgroup_summary.csv
 - JSON dir: /data2/Akbar1/sleep_stages_Dibatic/shhs_sleepstaging_planA/MESA_subgroup_analysis_outputs/mesa_subgroup_json
 - Plot dir: /data2/Akbar1/sleep_stages_Dibatic/shhs_sleepstaging_planA/MESA_subgroup_analysis_outputs/mesa_subgroup_plots


,dataset,axis,group,N_epochs,acc,macro_f1,kappa,AUROC,AUPRC,ECE,meanConf,bad_batches,csv,json
0,MESA,age,group_adult,829767,0.775171,0.635060,0.676426,0.922762,0.725576,0.110496,0.664686,0,/data2/Akbar1/sleep_stages_Dibatic/shhs_sleeps...,/data2/Akbar1/sleep_stages_Dibatic/shhs_sleeps...
1,MESA,age,group_senior,1380750,0.769117,0.625717,0.665713,0.919880,0.710051,0.121425,0.647692,0,/data2/Akbar1/sleep_stages_Dibatic/shhs_sleeps...,/data2/Akbar1/sleep_stages_Dibatic/shhs_sleeps...
2,MESA,ahi,severity_mild,709739,0.780934,0.635003,0.683615,0.925576,0.725243,0.118328,0.662606,0,/data2/Akbar1/sleep_stages_Dibatic/shhs_sleeps...,/data2/Akbar1/sleep_stages_Dibatic/shhs_sleeps...
3,MESA,ahi,severity_moderate,566325,0.770146,0.631452,0.667931,0.918964,0.710530,0.120289,0.649857,0,/data2/Akbar1/sleep_stages_Dibatic/shhs_sleeps...,/data2/Akbar1/sleep_stages_Dibatic/shhs_sleeps...
4,MESA,ahi,severity_no_osa,449499,0.787577,0.644222,0.691999,0.926333,0.725160,0.119313,0.668282,0,/data2/Akbar1/sleep_stages_Dibatic/shhs_sleeps...,/data2/Akbar1/sleep_stages_Dibatic/shhs_sleeps...
5,MESA,ahi,severity_severe,484954,0.743868,0.601742,0.630614,0.910362,0.693828,0.110542,0.633327,0,/data2/Akbar1/sleep_stages_Dibatic/shhs_sleeps...,/data2/Akbar1/sleep_stages_Dibatic/shhs_sleeps...
6,MESA,bmi,group_normal,612587,0.777937,0.638496,0.680047,0.925968,0.727576,0.118077,0.659860,0,/data2/Akbar1/sleep_stages_Dibatic/shhs_sleeps...,/data2/Akbar1/sleep_stages_Dibatic/shhs_sleeps...
7,MESA,bmi,group_obese,755145,0.764971,0.621709,0.659894,0.914743,0.704337,0.113445,0.651526,0,/data2/Akbar1/sleep_stages_Dibatic/shhs_sleeps...,/data2/Akbar1/sleep_stages_Dibatic/shhs_sleeps...
8,MESA,bmi,group_overweight,838207,0.772513,0.629312,0.671500,0.922950,0.718467,0.120347,0.652166,0,/data2/Akbar1/sleep_stages_Dibatic/shhs_sleeps...,/data2/Akbar1/sleep_stages_Dibatic/shhs_sleeps...
9,MESA,hypertension,normotensive,1355688,0.772938,0.633483,0.673915,0.921758,0.720951,0.116574,0.656364,0,/data2/Akbar1/sleep_stages_Dibatic/shhs_sleeps...,/data2/Akbar1/sleep_stages_Dibatic/shhs_sleeps...


In [25]:
# ============================================================
# CELL M4: Beautiful MESA subgroup results table (print + save)
# - Loads mesa_df (from CELL M3) or reloads from CSV
# - Pretty console display
# - Saves: Excel + styled HTML (paper-ready)
# ============================================================
from pathlib import Path
import pandas as pd
import numpy as np

OUT_DIR = Path("/data2/Akbar1/sleep_stages_Dibatic/shhs_sleepstaging_planA/MESA_subgroup_analysis_outputs")
mesa_csv = OUT_DIR / "MESA_subgroup_summary.csv"

# Load mesa_df if not in memory
if "mesa_df" not in globals() or mesa_df is None or len(mesa_df) == 0:
    assert mesa_csv.exists(), f"Missing: {mesa_csv}"
    mesa_df = pd.read_csv(mesa_csv)

# ---------- helpers ----------
def _fmt(x, nd=4):
    if pd.isna(x):
        return "NA"
    try:
        return f"{float(x):.{nd}f}"
    except Exception:
        return str(x)

def _axis_order(ax):
    # keeps common ones near top, others after
    order = {
        "sex": 0, "age": 1, "age_group": 1, "age_bin": 1,
        "bmi": 2, "bmi_group": 2,
        "ahi": 3, "ahi_severity": 3,
        "hypertension": 4, "htn": 4,
    }
    return order.get(str(ax).lower(), 99)

# ensure numeric
for col in ["acc","macro_f1","kappa","AUROC","AUPRC","ECE","meanConf"]:
    if col in mesa_df.columns:
        mesa_df[col] = pd.to_numeric(mesa_df[col], errors="coerce")

mesa_df["axis_order"] = mesa_df["axis"].apply(_axis_order)
mesa_df = mesa_df.sort_values(["axis_order","axis","group"]).drop(columns=["axis_order"]).reset_index(drop=True)

# ============================================================
# (A) PAPER-READY SUMMARY BY AXIS (mean ± std, plus min/max)
# ============================================================
agg = (mesa_df
       .groupby("axis")
       .agg(
           groups=("group","count"),
           N_epochs=("N_epochs","sum"),
           macro_f1_mean=("macro_f1","mean"),
           macro_f1_std=("macro_f1","std"),
           acc_mean=("acc","mean"),
           acc_std=("acc","std"),
           kappa_mean=("kappa","mean"),
           kappa_std=("kappa","std"),
           ece_mean=("ECE","mean"),
           ece_std=("ECE","std"),
           conf_mean=("meanConf","mean"),
           conf_std=("meanConf","std"),
       )
       .reset_index()
      )

# nice formatted columns
agg_disp = agg.copy()
agg_disp["macro_f1 (mean±std)"] = agg.apply(lambda r: f"{_fmt(r.macro_f1_mean)} ± {_fmt(r.macro_f1_std)}", axis=1)
agg_disp["acc (mean±std)"]      = agg.apply(lambda r: f"{_fmt(r.acc_mean)} ± {_fmt(r.acc_std)}", axis=1)
agg_disp["kappa (mean±std)"]    = agg.apply(lambda r: f"{_fmt(r.kappa_mean)} ± {_fmt(r.kappa_std)}", axis=1)
agg_disp["ECE (mean±std)"]      = agg.apply(lambda r: f"{_fmt(r.ece_mean)} ± {_fmt(r.ece_std)}", axis=1)
agg_disp["meanConf (mean±std)"] = agg.apply(lambda r: f"{_fmt(r.conf_mean)} ± {_fmt(r.conf_std)}", axis=1)

agg_disp = agg_disp[["axis","groups","N_epochs","macro_f1 (mean±std)","acc (mean±std)","kappa (mean±std)","ECE (mean±std)","meanConf (mean±std)"]]
print("\n==================== MESA: SUMMARY BY AXIS ====================")
display(agg_disp)

# ============================================================
# (B) FULL SUBGROUP TABLE (sorted, formatted)
# ============================================================
sub_disp = mesa_df.copy()

# create "compact" formatted strings
sub_disp["acc"]      = sub_disp["acc"].map(lambda x: _fmt(x, 4))
sub_disp["macro_f1"]  = sub_disp["macro_f1"].map(lambda x: _fmt(x, 4))
sub_disp["kappa"]    = sub_disp["kappa"].map(lambda x: _fmt(x, 4))
sub_disp["AUROC"]    = sub_disp["AUROC"].map(lambda x: _fmt(x, 4))
sub_disp["AUPRC"]    = sub_disp["AUPRC"].map(lambda x: _fmt(x, 4))
sub_disp["ECE"]      = sub_disp["ECE"].map(lambda x: _fmt(x, 4))
sub_disp["meanConf"] = sub_disp["meanConf"].map(lambda x: _fmt(x, 4))

cols_keep = ["axis","group","N_epochs","acc","macro_f1","kappa","AUROC","AUPRC","ECE","meanConf","bad_batches"]
cols_keep = [c for c in cols_keep if c in sub_disp.columns]
sub_disp = sub_disp[cols_keep]

print("\n==================== MESA: ALL SUBGROUPS (DETAILED) ====================")
display(sub_disp)

# ============================================================
# (C) BEST/WORST SUBGROUPS (by macro-F1)
# ============================================================
mesa_num = mesa_df.copy()
mesa_num = mesa_num.dropna(subset=["macro_f1"]).reset_index(drop=True)

best = mesa_num.sort_values("macro_f1", ascending=False).head(5)[["axis","group","N_epochs","macro_f1","acc","kappa","ECE","meanConf"]]
worst = mesa_num.sort_values("macro_f1", ascending=True).head(5)[["axis","group","N_epochs","macro_f1","acc","kappa","ECE","meanConf"]]

print("\n==================== TOP-5 SUBGROUPS (by macro-F1) ====================")
display(best)

print("\n==================== BOTTOM-5 SUBGROUPS (by macro-F1) ====================")
display(worst)

# ============================================================
# (D) SAVE: Excel + Styled HTML (paper-ready)
# ============================================================
xlsx_path = OUT_DIR / "MESA_beautiful_tables.xlsx"
html_path = OUT_DIR / "MESA_beautiful_subgroup_table.html"

with pd.ExcelWriter(xlsx_path, engine="openpyxl") as writer:
    agg.to_excel(writer, index=False, sheet_name="axis_summary_raw")
    agg_disp.to_excel(writer, index=False, sheet_name="axis_summary_pretty")
    mesa_df.to_excel(writer, index=False, sheet_name="subgroups_raw")
    sub_disp.to_excel(writer, index=False, sheet_name="subgroups_pretty")
    best.to_excel(writer, index=False, sheet_name="top5")
    worst.to_excel(writer, index=False, sheet_name="bottom5")

# Styled HTML (highlights macro_f1 and acc)
style_df = mesa_df.copy()
for col in ["acc","macro_f1","kappa","AUROC","AUPRC","ECE","meanConf"]:
    if col in style_df.columns:
        style_df[col] = pd.to_numeric(style_df[col], errors="coerce")

styler = (style_df[["axis","group","N_epochs","acc","macro_f1","kappa","ECE","meanConf"]]
          .sort_values(["axis","group"])
          .style
          .format({
              "acc":"{:.4f}",
              "macro_f1":"{:.4f}",
              "kappa":"{:.4f}",
              "ECE":"{:.4f}",
              "meanConf":"{:.4f}",
          }, na_rep="NA")
          .background_gradient(subset=["macro_f1"], axis=0)   # auto colors
          .background_gradient(subset=["acc"], axis=0)
          .set_properties(**{"font-family":"Arial", "font-size":"12px"})
          .set_table_styles([
              {"selector":"th", "props":[("text-align","left"),("font-weight","bold")]},
              {"selector":"td", "props":[("text-align","left")]},
          ])
         )

html = styler.to_html()
with open(html_path, "w") as f:
    f.write(html)

print("\n? Saved tables:")
print(" - Excel :", xlsx_path)
print(" - HTML  :", html_path)



==================== MESA: SUMMARY BY AXIS ====================


,axis,groups,N_epochs,macro_f1 (mean±std),acc (mean±std),kappa (mean±std),ECE (mean±std),meanConf (mean±std)
0,age,2,2210517,0.6304 ± 0.0066,0.7721 ± 0.0043,0.6711 ± 0.0076,0.1160 ± 0.0077,0.6562 ± 0.0120
1,ahi,4,2210517,0.6281 ± 0.0184,0.7706 ± 0.0192,0.6685 ± 0.0272,0.1171 ± 0.0045,0.6535 ± 0.0155
2,bmi,3,2205939,0.6298 ± 0.0084,0.7718 ± 0.0065,0.6705 ± 0.0101,0.1173 ± 0.0035,0.6545 ± 0.0046
3,hypertension,2,2210517,0.6279 ± 0.0080,0.7709 ± 0.0028,0.6684 ± 0.0078,0.1175 ± 0.0014,0.6534 ± 0.0042
4,sex,2,2210517,0.6246 ± 0.0139,0.7717 ± 0.0035,0.6697 ± 0.0015,0.1183 ± 0.0128,0.6533 ± 0.0093



==================== MESA: ALL SUBGROUPS (DETAILED) ====================


,axis,group,N_epochs,acc,macro_f1,kappa,AUROC,AUPRC,ECE,meanConf,bad_batches
0,sex,female,1227429,0.7692,0.6344,0.6687,0.9194,0.7219,0.1093,0.6599,0
1,sex,male,983088,0.7741,0.6147,0.6708,0.9223,0.6976,0.1274,0.6467,0
2,age,group_adult,829767,0.7752,0.6351,0.6764,0.9228,0.7256,0.1105,0.6647,0
3,age,group_senior,1380750,0.7691,0.6257,0.6657,0.9199,0.7101,0.1214,0.6477,0
4,bmi,group_normal,612587,0.7779,0.6385,0.6800,0.9260,0.7276,0.1181,0.6599,0
5,bmi,group_obese,755145,0.7650,0.6217,0.6599,0.9147,0.7043,0.1134,0.6515,0
6,bmi,group_overweight,838207,0.7725,0.6293,0.6715,0.9230,0.7185,0.1203,0.6522,0
7,ahi,severity_mild,709739,0.7809,0.6350,0.6836,0.9256,0.7252,0.1183,0.6626,0
8,ahi,severity_moderate,566325,0.7701,0.6315,0.6679,0.9190,0.7105,0.1203,0.6499,0
9,ahi,severity_no_osa,449499,0.7876,0.6442,0.6920,0.9263,0.7252,0.1193,0.6683,0



==================== TOP-5 SUBGROUPS (by macro-F1) ====================


,axis,group,N_epochs,macro_f1,acc,kappa,ECE,meanConf
9,ahi,severity_no_osa,449499,0.644222,0.787577,0.691999,0.119313,0.668282
4,bmi,group_normal,612587,0.638496,0.777937,0.680047,0.118077,0.659860
2,age,group_adult,829767,0.635060,0.775171,0.676426,0.110496,0.664686
7,ahi,severity_mild,709739,0.635003,0.780934,0.683615,0.118328,0.662606
0,sex,female,1227429,0.634441,0.769209,0.668659,0.109264,0.659945



==================== BOTTOM-5 SUBGROUPS (by macro-F1) ====================


,axis,group,N_epochs,macro_f1,acc,kappa,ECE,meanConf
10,ahi,severity_severe,484954,0.601742,0.743868,0.630614,0.110542,0.633327
1,sex,male,983088,0.614739,0.774111,0.670779,0.127375,0.646736
5,bmi,group_obese,755145,0.621709,0.764971,0.659894,0.113445,0.651526
12,hypertension,unknown,854829,0.622236,0.768933,0.662931,0.118499,0.650433
3,age,group_senior,1380750,0.625717,0.769117,0.665713,0.121425,0.647692



? Saved tables:
 - Excel : /data2/Akbar1/sleep_stages_Dibatic/shhs_sleepstaging_planA/MESA_subgroup_analysis_outputs/MESA_beautiful_tables.xlsx
 - HTML  : /data2/Akbar1/sleep_stages_Dibatic/shhs_sleepstaging_planA/MESA_subgroup_analysis_outputs/MESA_beautiful_subgroup_table.html


In [27]:
# ============================================================
# CELL S4 (FIXED): Load SHHS2 subgroup summary robustly
# ============================================================

from pathlib import Path
import pandas as pd
import numpy as np

# ---- Candidate locations to search ----
SEARCH_ROOTS = [
    Path("/data2/Akbar1/sleep_stages_Dibatic/shhs_sleepstaging_planA"),
    Path("/data2/Akbar1/sleep_stages_Dibatic"),
]

found_csv = None

for root in SEARCH_ROOTS:
    for p in root.rglob("*SHHS2*subgroup*summary*.csv"):
        found_csv = p
        break
    if found_csv:
        break

# ---- Load dataframe ----
if "shhs2_df" in globals() and shhs2_df is not None and len(shhs2_df) > 0:
    print("? Using existing shhs2_df from memory")
elif found_csv is not None:
    print(f"? Found SHHS2 summary CSV:\n  {found_csv}")
    shhs2_df = pd.read_csv(found_csv)
else:
    raise FileNotFoundError(
        "? Could not find SHHS2 subgroup summary CSV.\n"
        "Make sure you saved it during SHHS2 subgroup evaluation."
    )

print("Rows:", len(shhs2_df))
print("Columns:", list(shhs2_df.columns))


? Found SHHS2 summary CSV:
  /data2/Akbar1/sleep_stages_Dibatic/shhs_sleepstaging_planA/subgroup_eval_outputs/SHHS2_subgroup_summary.csv
Rows: 14
Columns: ['axis', 'group', 'N_subjects', 'n_epochs', 'acc', 'macro_f1', 'kappa', 'AUROC', 'AUPRC', 'ECE', 'meanConf', 'json']


In [28]:
# ============================================================
# CELL S5: Beautiful SHHS2 subgroup results table
# ============================================================

def fmt(x, n=4):
    if pd.isna(x):
        return "NA"
    return f"{float(x):.{n}f}"

# Ensure numeric
for c in ["acc","macro_f1","kappa","AUROC","AUPRC","ECE","meanConf"]:
    if c in shhs2_df.columns:
        shhs2_df[c] = pd.to_numeric(shhs2_df[c], errors="coerce")

# Sort nicely
shhs2_df = shhs2_df.sort_values(["axis","group"]).reset_index(drop=True)

# ---- Pretty per-subgroup table ----
pretty = shhs2_df.copy()
for c in ["acc","macro_f1","kappa","AUROC","AUPRC","ECE","meanConf"]:
    if c in pretty.columns:
        pretty[c] = pretty[c].map(fmt)

cols = [
    "axis","group","N_epochs",
    "acc","macro_f1","kappa",
    "AUROC","AUPRC","ECE","meanConf",
    "bad_batches"
]
cols = [c for c in cols if c in pretty.columns]

pretty = pretty[cols]

print("\n==================== SHHS2 SUBGROUP PERFORMANCE ====================")
display(pretty)



==================== SHHS2 SUBGROUP PERFORMANCE ====================


,axis,group,acc,macro_f1,kappa,AUROC,AUPRC,ECE,meanConf
0,age,group_adult,0.8775,0.8157,0.8319,0.9751,0.8735,0.1741,0.7034
1,age,group_senior,0.8636,0.7936,0.8098,0.9728,0.8523,0.1787,0.6850
2,ahi,severity_mild,0.8738,0.8091,0.8263,0.9749,0.8653,0.1765,0.6973
3,ahi,severity_moderate,0.8627,0.7921,0.8089,0.9726,0.8520,0.1743,0.6884
4,ahi,severity_no_osa,0.8850,0.8216,0.8426,0.9799,0.8793,0.1789,0.7061
5,ahi,severity_severe,0.8461,0.7745,0.7810,0.9640,0.8380,0.1777,0.6684
6,bmi,group_normal,0.8741,0.8086,0.8266,0.9763,0.8667,0.1787,0.6954
7,bmi,group_obese,0.8668,0.7994,0.8149,0.9728,0.8585,0.1780,0.6888
8,bmi,group_overweight,0.8690,0.8022,0.8186,0.9734,0.8610,0.1746,0.6944
9,diabetes,diabetes,0.8648,0.7967,0.8120,0.9739,0.8576,0.1762,0.6885


In [31]:
# ============================================================
# CELL S6 (FIXED): Axis-level summary (auto column mapping)
# ============================================================

import pandas as pd
import numpy as np

def fmt(x, n=4):
    if pd.isna(x):
        return "NA"
    return f"{float(x):.{n}f}"

def pick_col(df, candidates, required=False, default=None):
    for c in candidates:
        if c in df.columns:
            return c
    if required:
        raise KeyError(f"None of these columns exist: {candidates}\nAvailable: {list(df.columns)}")
    return default

# ---- Detect columns robustly ----
col_axis  = pick_col(shhs2_df, ["axis"], required=True)
col_group = pick_col(shhs2_df, ["group"], required=True)

col_N = pick_col(shhs2_df, ["N_epochs","n_epochs","N","n","num_epochs","num_samples","N_samples"], required=False, default=None)
col_conf = pick_col(shhs2_df, ["meanConf","mean_conf","avg_conf","confidence","conf"], required=False, default=None)

# Metrics that should exist
metric_cols = {
    "macro_f1": pick_col(shhs2_df, ["macro_f1","macroF1","f1_macro"], required=True),
    "acc":      pick_col(shhs2_df, ["acc","accuracy"], required=True),
    "kappa":    pick_col(shhs2_df, ["kappa"], required=True),
    "ECE":      pick_col(shhs2_df, ["ECE","ece"], required=True),
}

# Make sure numerics are numeric
for k, c in metric_cols.items():
    shhs2_df[c] = pd.to_numeric(shhs2_df[c], errors="coerce")

if col_N is not None:
    shhs2_df[col_N] = pd.to_numeric(shhs2_df[col_N], errors="coerce")

if col_conf is not None:
    shhs2_df[col_conf] = pd.to_numeric(shhs2_df[col_conf], errors="coerce")

# ---- Build aggregation dict ----
agg = {
    "groups": (col_group, "count"),
    "macro_f1_mean": (metric_cols["macro_f1"], "mean"),
    "macro_f1_std":  (metric_cols["macro_f1"], "std"),
    "acc_mean":      (metric_cols["acc"], "mean"),
    "acc_std":       (metric_cols["acc"], "std"),
    "kappa_mean":    (metric_cols["kappa"], "mean"),
    "kappa_std":     (metric_cols["kappa"], "std"),
    "ece_mean":      (metric_cols["ECE"], "mean"),
}

if col_N is not None:
    agg["N_total"] = (col_N, "sum")

if col_conf is not None:
    agg["conf_mean"] = (col_conf, "mean")

summary = (
    shhs2_df
    .groupby(col_axis)
    .agg(**agg)
    .reset_index()
)

summary["macro-F1 (mean±std)"] = summary.apply(
    lambda r: f"{fmt(r.macro_f1_mean)} ± {fmt(r.macro_f1_std)}", axis=1
)
summary["Accuracy (mean±std)"] = summary.apply(
    lambda r: f"{fmt(r.acc_mean)} ± {fmt(r.acc_std)}", axis=1
)
summary["Kappa (mean±std)"] = summary.apply(
    lambda r: f"{fmt(r.kappa_mean)} ± {fmt(r.kappa_std)}", axis=1
)

# Display columns nicely
show_cols = [col_axis, "groups"]
if "N_total" in summary.columns: show_cols.append("N_total")
show_cols += ["macro-F1 (mean±std)", "Accuracy (mean±std)", "Kappa (mean±std)", "ece_mean"]
if "conf_mean" in summary.columns: show_cols.append("conf_mean")

print("\n==================== SHHS2 AXIS-LEVEL SUMMARY ====================")
display(summary[show_cols].sort_values(col_axis))



==================== SHHS2 AXIS-LEVEL SUMMARY ====================


,axis,groups,N_total,macro-F1 (mean±std),Accuracy (mean±std),Kappa (mean±std),ece_mean,conf_mean
0,age,2,2858985,0.8046 ± 0.0156,0.8706 ± 0.0098,0.8209 ± 0.0156,0.176374,0.694206
1,ahi,4,2858985,0.7993 ± 0.0205,0.8669 ± 0.0166,0.8147 ± 0.0263,0.176844,0.690061
2,bmi,3,2835633,0.8034 ± 0.0047,0.8700 ± 0.0037,0.8200 ± 0.0060,0.177101,0.692859
3,diabetes,2,2858985,0.8034 ± 0.0096,0.8700 ± 0.0074,0.8200 ± 0.0113,0.176778,0.693177
4,hypertension,1,2858985,0.8029 ± NA,0.8695 ± NA,0.8194 ± NA,0.176728,0.692774
5,sex,2,2858985,0.7997 ± 0.0214,0.8686 ± 0.0138,0.8171 ± 0.0238,0.176492,0.692105


In [32]:
# ============================================================
# CELL S5 (ROBUST): Beautiful per-subgroup table
# ============================================================

def fmt(x, n=4):
    if pd.isna(x):
        return "NA"
    return f"{float(x):.{n}f}"

col_N = pick_col(shhs2_df, ["N_epochs","n_epochs","N","n","num_epochs","num_samples","N_samples"], required=False, default=None)
col_conf = pick_col(shhs2_df, ["meanConf","mean_conf","avg_conf","confidence","conf"], required=False, default=None)

cols = ["axis", "group"]
if col_N is not None: cols.append(col_N)

for c in ["acc","macro_f1","kappa","AUROC","AUPRC","ECE"]:
    if c in shhs2_df.columns:
        shhs2_df[c] = pd.to_numeric(shhs2_df[c], errors="coerce")
        cols.append(c)

if col_conf is not None:
    cols.append(col_conf)

if "bad_batches" in shhs2_df.columns:
    cols.append("bad_batches")

pretty = shhs2_df[cols].copy().sort_values(["axis","group"]).reset_index(drop=True)
for c in pretty.columns:
    if c not in ["axis","group"] and c != col_N:
        pretty[c] = pretty[c].map(fmt)

print("\n==================== SHHS2 SUBGROUP PERFORMANCE ====================")
display(pretty)



==================== SHHS2 SUBGROUP PERFORMANCE ====================


,axis,group,n_epochs,acc,macro_f1,kappa,AUROC,AUPRC,ECE,meanConf
0,age,group_adult,1207883,0.8775,0.8157,0.8319,0.9751,0.8735,0.1741,0.7034
1,age,group_senior,1651102,0.8636,0.7936,0.8098,0.9728,0.8523,0.1787,0.6850
2,ahi,severity_mild,1040788,0.8738,0.8091,0.8263,0.9749,0.8653,0.1765,0.6973
3,ahi,severity_moderate,687636,0.8627,0.7921,0.8089,0.9726,0.8520,0.1743,0.6884
4,ahi,severity_no_osa,684726,0.8850,0.8216,0.8426,0.9799,0.8793,0.1789,0.7061
5,ahi,severity_severe,445835,0.8461,0.7745,0.7810,0.9640,0.8380,0.1777,0.6684
6,bmi,group_normal,730333,0.8741,0.8086,0.8266,0.9763,0.8667,0.1787,0.6954
7,bmi,group_obese,913767,0.8668,0.7994,0.8149,0.9728,0.8585,0.1780,0.6888
8,bmi,group_overweight,1191533,0.8690,0.8022,0.8186,0.9734,0.8610,0.1746,0.6944
9,diabetes,diabetes,1553881,0.8648,0.7967,0.8120,0.9739,0.8576,0.1762,0.6885
